# Create the Element Model
This notebook is to create the element model to run the TARDIS again. 
However, you still need to determine the photosphere (inner shell) velocity and the requested luminosity to make a perfect-fitting spectra.  
An example of the Tardis configuration is given in "IGmodel/DD-Sprinter-Lumi8.67.yml", you may need to re-define the spectral-line file location, the SN explosion time, the random seed, the element abundance name.  
In the paper, I searched the best-fit manually mainly due to the limited computation facility. 
I recommend you to use grid-search to make it more efficiently.  
Also, when running the Tardis, don't forget to use the density profile in "IGmodel/Density.dat".  

In [3]:
import os
import glob
import yaml
import keras
import astropy
import numpy as np
import pandas as pd
import keras.backend as K
import astropy.units as u
import astropy.constants as c
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from keras.models import Sequential,Model
from dust_extinction.averages import GCC09_MWAvg
from sklearn.model_selection import train_test_split
from dust_extinction.parameter_averages import CCM89,F99

Using TensorFlow backend.


In [7]:
spectralist=[
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_0.4d.dat',
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_-2.6d.dat',
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_3.7d.dat',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_-3.1d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_-1.1d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_0.9d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_3.9d.flm',
    'ObserveSpectra/Prediction/SN2011iv/SN2011iv_0.6d.flm',
    'ObserveSpectra/Prediction/SN2015F/SN2015F_-2.3d.flm',
    'ObserveSpectra/Prediction/ASASSN-14lp/ASASSN-14lp_-4.4d.flm',
    'ObserveSpectra/Prediction/SN2011by/SN2011by_-0.4d.flm']
ext1list=[CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),
         CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),
         CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),F99(Rv=3.1)]
ext2list=[GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),
         GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),
         GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg()]
ext1EbvList=[0,0,0,0.206,0.206,0.206,0.206,0,0.035,0.33,0.039]
ext2EbvList=[0,0,0,0.135,0.135,0.135,0.135,0,0.175,0.021,0.013]
zlist=[0.000804,0.000804,0.000804,0.00389,0.00389,0.00389,0.00389,0.006494,0.0049,0.0051,0.002843]
snname=['SN2011fe_0.4d','SN2011fe_-2.6d','SN2011fe_3.7d','SN2013dy_-3.1d','SN2013dy_-1.1d',
        'SN2013dy_0.9d','SN2013dy_3.9d','SN2011iv_0.6d','SN2015F_-2.3d','ASASSN-14lp_-4.4d','SN2011by_-0.4d']

In [15]:
def WriteOneAbund(Element,chosenelem,zone,amplifier):
    if zone==1:Element[:19,chosenelem]=Element[:19,chosenelem]*amplifier
    if zone==2:Element[19:30,chosenelem]=Element[19:30,chosenelem]*amplifier
    if zone==3:Element[30:40,chosenelem]=Element[30:40,chosenelem]*amplifier
    if zone==4:Element[40:,chosenelem]=Element[40:,chosenelem]*amplifier
    return Element
def ElemWritter(elemdata,elemindex,elemname):
    Element=np.genfromtxt('IGmodel/Element.dat')
    for k in range(len(elemindex)):
        Element=WriteOneAbund(Element,\
                              int(elemindex[k].split('_')[0]),\
                              int(elemindex[k].split('_')[1]),\
                              elemdata[k])
    
    Element=pd.DataFrame(Element)
    Element[0]=pd.to_numeric(Element[0],downcast='integer')
    Element.to_csv(elemname,index=False,header=False,sep=' ')#elemname default is 'TdCache/Element.dat'
    
    with open(elemname,'r+') as f:
        old=f.read()
        f.seek(0)
        f.write('# index Z=1 - Z=30\n')
        f.write(old)
    return

## Read the Multiplication Factor
Don't forget to visit the "ExecutePredict.ipynb" before that.  
In this notebook, it reads the multiplication factor (which is the output of the other notebook) then convert it to Tardis's element abundance profile.  

In [9]:
RefineData=pd.read_csv('DataCache/2019.3.3/RefineData.csv',index_col=0)

In [13]:
ProberRes=pd.read_csv('MdSaver/SmallRun/DataRes.csv')
RawElems=[str(i)+'_'+str(j) for i in range(6,29) for j in [1,2,3,4]]
ChosenElem=[]
for k in RawElems:
    if ProberRes[k][0]>0.1:continue
    ChosenElem.append(k)

In [18]:
for i in range(len(RefineData)):
    ElemAmplify=np.array(RefineData.iloc[i,1:]).tolist()
    ElemAmplify=np.array(ElemAmplify)
    ElemAmplify=(abs(ElemAmplify)+ElemAmplify)/2
    ElemWritter(elemdata=ElemAmplify,elemindex=ChosenElem,elemname='TdCache/PredElemsAll4/'+RefineData['NameList'][i]+'.abund.dat')  